In [1]:
import numpy as np
import pandas as pd
import requests
import datetime
import time

import env

In [2]:
baseUrl = 'https://www.googleapis.com/youtube/v3/search'
youtubeDataBaseParams = {
  'key': env.token,
  'part': 'snippet',
  'order': 'viewCount',
  'regionCode': 'KR',
  'type': 'video'
}

numOfVideoPerMovie = 5
searchKeywords = ['영화', '리뷰', '해석', '의미', '명장면']

In [4]:
movieInfo = pd.read_csv('movieInfo.csv').loc[:,['영화명', '개봉일']]
movieNames = list(movieInfo.loc[:, '영화명'])

In [6]:
columns = ['영화 제목', '검색 키워드', '영상 id', '영상 제목', '생성된 날짜']
res_data = pd.DataFrame(columns=columns)

for movieName in movieNames[:9]:
  for keyword in searchKeywords:
    publishedAfter = list(movieInfo[movieInfo['영화명'] == movieName].loc[:, '개봉일'])[0]
    year, month, day = publishedAfter.split('-')
    publishedBefore = (datetime.datetime(int(year), int(month), int(day)) + datetime.timedelta(days=30)).strftime('%Y-%m-%d')
    q = ' '.join([movieName, keyword])

    youtubeDataBaseParams['q'] = q
    youtubeDataBaseParams['publishedAfter'] = publishedAfter+'T00:00:00Z'
    youtubeDataBaseParams['publishedBefore'] = publishedBefore+'T00:00:00Z'

    res = requests.get(baseUrl, youtubeDataBaseParams)
    data = res.json()

    for item in data['items']:
      item_data = [[movieName, q, item['id']['videoId'], item['snippet']['title'], item['snippet']['publishTime']]]
      res_data = res_data.append(pd.DataFrame(item_data, columns=columns),ignore_index=True)

  time.sleep(20)

res_data
    

,영화 제목,검색 키워드,영상 id,영상 제목,생성된 날짜
0,극한직업,극한직업 영화,YcMZZ3sjfxk,"극한직업, 닭으로 닥치고 천만 찍어버릴 영화!",2019-02-02T01:01:58Z
1,극한직업,극한직업 영화,7DRDMuMdxVY,극한직업 수원왕갈비통닭을 먹었습니다.맛상무,2019-02-10T05:45:02Z
2,극한직업,극한직업 영화,8iQg4napg90,영화 &quot;극한직업&quot;의 촬영 비하인드 스토리,2019-02-03T05:19:01Z
3,극한직업,극한직업 영화,jFd1X9j_9Os,명절 개봉영화의 역사를 새로쓰는 영화 : &#39;극한직업&#39;리뷰,2019-01-31T13:02:53Z
4,극한직업,극한직업 영화,-iHkdgvLBlY,웃다 지린다고 소문난 극한직업 보고 만든 웃긴 영화 볼 때 갑분싸 유형ㅋㅋㅋㅋㅋㅋㅋㅋ,2019-01-25T08:46:13Z
...,...,...,...,...,...
187,나쁜 녀석들: 더 무비,나쁜 녀석들: 더 무비 명장면,8qROF7VtUQE,"[#뭅뭅픽] 누군가는 겪었을 이야기, 국제시장 속 명장면 명대사 모음",2019-09-13T01:00:02Z
188,나쁜 녀석들: 더 무비,나쁜 녀석들: 더 무비 명장면,6yzjoR91LKQ,[나쁜녀석들] 형한테 혼나볼래? 美친개의 시작 마동석X김상중 레전드 하이라이트 모음...,2019-09-11T01:00:00Z
189,나쁜 녀석들: 더 무비,나쁜 녀석들: 더 무비 명장면,rUXS2LuArpI,나쁜 녀석들 : 더 무비 _ 🍺영화 리뷰,2019-09-12T03:30:13Z
190,나쁜 녀석들: 더 무비,나쁜 녀석들: 더 무비 명장면,i9zAyewGwlE,[나쁜 녀석들: 더 무비] &#39;4인 4색 액션&#39; 영상,2019-09-16T09:56:15Z


In [7]:
res_data.to_csv('./movieVideoList.csv')